In [1]:
# title: SoilData - Soil Organic Carbon Stock
# subtitle: Prepare environmental covariates
# author: Alessandro Samuel-Rosa and Taciara Zborowski Horst
# data: 2024 CC-BY

# Import necessary libraries
import os
import pandas as pd

# Identify working directory, saving the path to a variable
src_dir = os.getcwd()
work_dir = os.path.dirname(src_dir)

# Read the TXT file '20_soildata_soc.txt' with the soil data from the 'data' folder
# Field separator: tab
file_path = os.path.join(work_dir, 'data', '20_soildata_soc.txt')
soildata_df = pd.read_csv(file_path, sep='\t', low_memory=False)
print(soildata_df.shape)

# Read the TXT file '21_soildata_soc.txt' with the coordinates from the 'data' folder
# Field separator: tab. Decimal separator: comma
file_path = os.path.join(work_dir, 'data', '21_soildata_soc.txt')
df2 = pd.read_csv(file_path, sep='\t', low_memory=False)
print(df2.shape)

# Compare the two data frames to check if there were changes in the data
# Use equals() method to compare the two data frames
# The result is a boolean value, True if the data frames are equal, False otherwise
# We consider only the following variables in the comparison:
# dataset_id, observacao_id, coord_x, coord_y, data_ano
df1 = soildata_df[['dataset_id', 'observacao_id', 'coord_x', 'coord_y', 'data_ano']]
df2 = df2[['dataset_id', 'observacao_id', 'coord_x', 'coord_y', 'data_ano']]
if df1.equals(df2):
    print("The DataFrames are identical.")
else:
    print("The DataFrames are different.")
del df1, df2
# 29683 layers (GET FROM PREVIOUS SCRIPT)

(29881, 85)
(29683, 122)
The DataFrames are different.


In [2]:
# Filter out soil layers without geographic coordinates
# coord_x and coord_y are the columns with the geographic coordinates
soildata_xy = soildata_df[soildata_df['coord_x'].notnull()]
soildata_xy = soildata_xy[soildata_xy['coord_y'].notnull()]

# Remove all duplicates based on the following columns:
# "dataset_id", "observacao_id", "coord_x", "coord_y", "data_ano"
# The first occurrence is kept, and the others are removed
soildata_xy = soildata_xy[['dataset_id', 'observacao_id', 'coord_x', 'coord_y', 'data_ano']]
soildata_xy = soildata_xy.drop_duplicates()
target = 13292 # GET VALUE FROM PREVIOUS SCRIPT!
print(
  'There should be', target, 'events:', target == soildata_xy.shape[0],
  '\nThere are', soildata_xy.shape[0], 'events')

There should be 13292 events: False 
There are 13381 events


In [3]:
# Print the first 5 rows of the data frame
print(soildata_xy)

      dataset_id observacao_id    coord_x    coord_y  data_ano
0        ctb0003   sm-dnos-001 -53.794645 -29.651271    2009.0
1        ctb0003   sm-dnos-002 -53.793987 -29.650564    2009.0
2        ctb0003   sm-dnos-003 -53.793993 -29.650232    2009.0
3        ctb0003   sm-dnos-004 -53.794345 -29.650311    2009.0
4        ctb0003   sm-dnos-005 -53.792947 -29.650838    2009.0
...          ...           ...        ...        ...       ...
29869    ctb0838            74 -43.997580 -22.564130    2014.0
29871    ctb0838            75 -43.994350 -22.522300    2014.0
29873    ctb0838            78 -43.989760 -22.530160    2014.0
29876    ctb0838            88 -44.007310 -22.531180    2014.0
29879    ctb0838            89 -44.006970 -22.530190    2014.0

[13381 rows x 5 columns]


In [4]:
# Import necessary libraries
import ee
import geemap

# GEE user authentication
ee.Authenticate()

# Initialize the Earth Engine API
ee.Initialize()

# Convert DataFrame to Earth Engine Feature Collection
soildata_fc = geemap.df_to_ee(soildata_xy, latitude = 'coord_y', longitude = 'coord_x')
print(soildata_fc.size().getInfo(), 'features')

# Function to split a feature collection (sampling points) into chunks
def split_sampling_points(fc, chunk_size):
    features = fc.toList(fc.size())
    chunks = [features.slice(i, i + chunk_size) for i in range(0, features.size().getInfo(), chunk_size)]
    return [ee.FeatureCollection(chunk) for chunk in chunks]

# Split the sample points into subsets of 1000 points each
# This is necessary to avoid timeout errors
chunk_size = 1000
soildata_chunks = split_sampling_points(soildata_fc, chunk_size)
print(len(soildata_chunks), 'chunks')


Successfully saved authorization token.


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


13381 features
14 chunks


In [5]:
# Soil Grids 250m v2.0
# (this takes about 10 minutes to run)

# Soil Grids 250m v2.0: bdod_mean (bulk density)
bdod_image = ee.Image("projects/soilgrids-isric/bdod_mean")
bdod_image = bdod_image.select(['bdod_0-5cm_mean', 'bdod_5-15cm_mean', 'bdod_15-30cm_mean'])

# Soil Grids 250m v2.0: clay_mean (clay content)
clay_image = ee.Image("projects/soilgrids-isric/clay_mean")
clay_image = clay_image.select(['clay_0-5cm_mean', 'clay_5-15cm_mean', 'clay_15-30cm_mean'])

# Soil Grids 250m v2.0: sand_mean (sand content)
sand_image = ee.Image("projects/soilgrids-isric/sand_mean")
sand_image = sand_image.select(['sand_0-5cm_mean', 'sand_5-15cm_mean', 'sand_15-30cm_mean'])

# Soil Grids 250m v2.0: soc_mean (soil organic carbon)
soc_image = ee.Image("projects/soilgrids-isric/soc_mean")
soc_image = soc_image.select(['soc_0-5cm_mean', 'soc_5-15cm_mean', 'soc_15-30cm_mean'])

# Soil Grids 250m v2.0: cfvo_mean (coarse fragments volume)
cfvo_image = ee.Image("projects/soilgrids-isric/cfvo_mean")
cfvo_image = cfvo_image.select(['cfvo_0-5cm_mean', 'cfvo_5-15cm_mean', 'cfvo_15-30cm_mean'])

# Stack the images into a single multiband image
soilgrids_image = bdod_image.addBands([clay_image, sand_image, soc_image, cfvo_image])

# Initialize an empty list to store DataFrames
dataframes = []

# Loop over each subset and sample the data
for chunk in soildata_chunks:
    sampled_points = geemap.extract_values_to_points(chunk, soilgrids_image, scale=250)
    sampled_df = geemap.ee_to_df(sampled_points)
    dataframes.append(sampled_df)

# Concatenate all DataFrames into a single DataFrame
soilgrids_df = pd.concat(dataframes, ignore_index=True)

# Rename columns
# Replace dash with underscores
soilgrids_df.columns = soilgrids_df.columns.str.replace('-', '_')
# Remove _mean from column names
soilgrids_df.columns = soilgrids_df.columns.str.replace('_mean', '')

# Print column names
print(soilgrids_df.columns)

# Check if number of rows of soildata_df is the same as that of soildata_xy
# If the number of rows is the same, the merge was successful
target = soildata_xy.shape[0]
print(
  '\nThere should be', target, 'events:',
  target == soilgrids_df.shape[0], '\nThere are', soilgrids_df.shape[0], 'events')

Index(['bdod_0_5cm', 'bdod_15_30cm', 'bdod_5_15cm', 'cfvo_0_5cm',
       'cfvo_15_30cm', 'cfvo_5_15cm', 'clay_0_5cm', 'clay_15_30cm',
       'clay_5_15cm', 'coord_x', 'coord_y', 'data_ano', 'dataset_id',
       'observacao_id', 'sand_0_5cm', 'sand_15_30cm', 'sand_5_15cm',
       'soc_0_5cm', 'soc_15_30cm', 'soc_5_15cm'],
      dtype='object')

There should be 13381 events: True 
There are 13381 events


In [6]:
# Compute summary statistics of the SoilGrids data
summary = soilgrids_df.describe()
print(summary)

         bdod_0_5cm  bdod_15_30cm   bdod_5_15cm    cfvo_0_5cm  cfvo_15_30cm  \
count  13086.000000  13086.000000  13086.000000  13102.000000  13102.000000   
mean     117.701895    126.053187    121.594299     62.542589     66.840788   
std       11.336127     10.724247     11.060162     33.305375     36.243606   
min       61.000000     60.000000     60.000000      0.000000      0.000000   
25%      111.000000    121.000000    116.000000     36.000000     39.000000   
50%      118.000000    128.000000    123.000000     58.000000     61.000000   
75%      126.000000    134.000000    129.000000     86.000000     93.000000   
max      145.000000    158.000000    148.000000    364.000000    336.000000   

        cfvo_5_15cm    clay_0_5cm  clay_15_30cm   clay_5_15cm       coord_x  \
count  13102.000000  13102.000000  13102.000000  13102.000000  13381.000000   
mean      63.853076    310.440925    345.596626    313.234239    -52.668279   
std       35.432377    105.257271    100.442192    

In [7]:
from datetime import datetime

# MapBiomas LULC Collection
# (this takes about 10 minutes to run)

# Import the MapBiomas Collection 9.0
collection = 'projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1'
mapbiomas_image = ee.Image(collection)

# Initialize an empty list to store DataFrames
dataframes = []

# Loop over each subset and sample the data
for chunk in soildata_chunks:
    sampled_points = geemap.extract_values_to_points(chunk, mapbiomas_image, scale=30)
    sampled_df = geemap.ee_to_df(sampled_points)
    dataframes.append(sampled_df)

In [8]:
# Concatenate all DataFrames into a single DataFrame
mapbiomas_df = pd.concat(dataframes, ignore_index=True)

# Rename columns
# Remove classification_ prefix from column names
mapbiomas_df.columns = mapbiomas_df.columns.str.replace('classification_', '')

# Get LULC class at the year of sampling (data_ano)
# Each column in the MapBiomas dataset represents a year. The column name is the year of the
# classification. The value is the class code. We need to extract the class code for the year of
# sampling, which is stored in the data_ano column.
# Step 1: Create a new column YEAR based on data_ano
mapbiomas_df['YEAR'] = mapbiomas_df['data_ano']
# Step 2: If YEAR is less than 1985, set it to 0 (no data)
mapbiomas_df.loc[mapbiomas_df['YEAR'] < 1985, 'YEAR'] = 0
# Step 3: Find the column index for each YEAR
lulc_idx = mapbiomas_df.columns.get_indexer(mapbiomas_df['YEAR'].astype(str))
# Step 4: Extract the class code for each row based on the data_ano column
lulc = mapbiomas_df.to_numpy()
lulc = lulc[range(len(lulc)), lulc_idx]
# Step 5: Convert the extracted class codes to strings and assign them to a new column lulc
mapbiomas_df['lulc'] = lulc.astype(str)
# Step 6: Drop the YEAR column
mapbiomas_df = mapbiomas_df.drop(columns = ['YEAR'])

# Some columns of mapbiomas_df are named with the year of the classification, ranging from 1985 to
# the present year. This columns need to be dropped.
# Drop columns with years as column names
current_year = datetime.now().year
years_to_drop = [str(year) for year in range(1985, current_year + 1)]
mapbiomas_df.drop(columns=years_to_drop, inplace=True, errors='ignore')

# Reclassify the land cover classes
forest_codes = ['1.0', '3.0', '4.0', '5.0', '6.0', '49.0']
nonforest_codes = ['10.0', '11.0', '12.0', '32.0', '29.0', '50.0']
pasture_codes = ['15.0']
agriculture_codes = [
    '14.0', '18.0', '19.0', '39.0', '20.0', '40.0', '62.0', '41.0', '36.0', '46.0', '47.0', '48.0',
    '21.0', '35.0']
forestry_codes = ['9.0']
nonvegetation_codes = ['22.0', '23.0', '24.0', '30.0', '25.0', '26.0', '33.0', '31.0', '27.0']
na_codes = ['0', '0.0', 'nan']
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(forest_codes, 'forest')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(nonforest_codes, 'nonforest')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(pasture_codes, 'pasture')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(agriculture_codes, 'agriculture')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(forestry_codes, 'forestry')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(nonvegetation_codes, 'nonvegetation')
mapbiomas_df['lulc'] = mapbiomas_df['lulc'].replace(na_codes, pd.NA)

# Print summary of the land cover classes, including NA
print('\nDistribution of land cover/land use classes (including NA):')
print(mapbiomas_df['lulc'].value_counts(dropna=False))

# Check if number of rows of mapbiomas_df is the same as that of soildata_xy
# If the number of rows is the same, the sampling was successful
target = soildata_xy.shape[0]
print(
  '\nThere should be', target, 'events:',
  target == mapbiomas_df.shape[0], '\nThere are', mapbiomas_df.shape[0], 'events')


Distribution of land cover/land use classes (including NA):
lulc
<NA>             4236
forest           3354
pasture          2667
agriculture      1690
nonforest        1023
nonvegetation     211
forestry          189
2024               11
Name: count, dtype: int64

There should be 13381 events: True 
There are 13381 events


In [9]:
# Geomorphometry 90 m

# Sample the terrain attributes
# projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/OT_GEOMORPHOMETRY_90m
# (this takes about 10 minutes to run)
dem = ee.Image("projects/mapbiomas-workspace/SOLOS/COVARIAVEIS/OT_GEOMORPHOMETRY_90m")

# Initialize an empty list to store DataFrames
dem_dfs = []

# Loop over each subset and sample the data
for chunk in soildata_chunks:
    sampled_points = geemap.extract_values_to_points(chunk, dem, scale=90)
    sampled_df = geemap.ee_to_df(sampled_points)
    dem_dfs.append(sampled_df)

# Concatenate all DataFrames into a single DataFrame
dem_dataframe = pd.concat(dem_dfs, ignore_index=True)

# print column names
print(dem_dataframe.columns)

Index(['aspect', 'aspect_cosine', 'aspect_sine', 'convergence', 'coord_x',
       'coord_y', 'cti', 'data_ano', 'dataset_id', 'dev_magnitude',
       'dev_scale', 'dx', 'dxx', 'dxy', 'dy', 'dyy', 'eastness', 'elev_stdev',
       'geom', 'northness', 'observacao_id', 'pcurv', 'rough_magnitude',
       'roughness', 'slope', 'spi'],
      dtype='object')


In [10]:
# Compute summary statistics of the Geomorphometry data
summary = dem_dataframe.describe()
print(summary)

             aspect  aspect_cosine   aspect_sine   convergence       coord_x  \
count  13377.000000   13347.000000  13377.000000  13377.000000  13381.000000   
mean     182.953342      -0.020292     -0.019612     -2.944035    -52.668279   
std      102.680984       0.656210      0.653156     25.632147      8.480355   
min        0.074554      -0.999748     -0.999727    -85.449432    -73.783330   
25%       95.621498      -0.650972     -0.635149    -19.795198    -60.957222   
50%      186.064224      -0.047982     -0.049310     -3.478558    -53.581038   
75%      269.087708       0.607631      0.607330     14.010069    -45.800000   
max      359.995667       0.999829      0.999761     83.113701    -34.883300   

            coord_y           cti      data_ano  dev_magnitude     dev_scale  \
count  13381.000000  13377.000000  13381.000000   13377.000000  13377.000000   
mean     -16.487456     -0.269968   1993.555863       0.321840    803.499439   
std        9.062978      2.175140     1

In [11]:
# Merge data sampled from SoilGrids, MapBiomas, and Geomorphometry into soildata_df
# Keep all rows from soildata_df
merge_columns = ['dataset_id', 'observacao_id', 'coord_x', 'coord_y', 'data_ano']
output_df = soildata_df.merge(soilgrids_df, on = merge_columns, how = 'left')
output_df = output_df.merge(mapbiomas_df, on = merge_columns, how = 'left')
output_df = output_df.merge(dem_dataframe, on = merge_columns, how = 'left')

# # Fill empty cells in the 'lulc' column with 'unknown'
# output_df['lulc'] = output_df['lulc'].fillna('unknown')

# Check if number of rows of output_df is the same as that of soildata_df
# If the number of rows is the same, the merge was successful
target = soildata_df.shape[0]
print(
  '\nThere should be', target, 'layers:',
  target == output_df.shape[0], '\nThere are', output_df.shape[0], 'layers')


There should be 29881 layers: True 
There are 29881 layers


In [12]:
# Write the output_df to a TXT file
# Field separator: tab. Decimal separator: comma
file_path = os.path.join(work_dir, 'data', '21_soildata_soc.txt')
output_df.to_csv(file_path, sep='\t', index=False)